# Testing save TF model and load with TFJS

## Save model from TF core

In [1]:
# Save
# ref: https://www.tensorflow.org/tutorials/keras/save_and_load#save_the_entire_model

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.0


In [3]:
# example dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 3s 0us/step


In [5]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training

In [7]:
# Save checkpoints during training
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - 1s 14ms/step - loss: 1.6062 - sparse_categorical_accuracy: 0.4808 - val_loss: 0.7302 - val_sparse_categorical_accuracy: 0.7820

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4862 - sparse_categorical_accuracy: 0.8543 - val_loss: 0.5412 - val_sparse_categorical_accuracy: 0.8300

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2919 - sparse_categorical_accuracy: 0.9209 - val_loss: 0.4707 - val_sparse_categorical_accuracy: 0.8550

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2063 - sparse_categorical_accuracy: 0.9578 - val_loss: 0.4318 - val_sparse_categorical_accuracy: 0.8710

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1699 - sparse_cat

In [8]:
# rebuild a fresh & untrained model + load weights later that
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3524 - sparse_categorical_accuracy: 0.0800
Untrained model, accuracy:  8.00%


In [9]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4037 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


In [10]:
# latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1/cp.ckpt'

In [11]:
# To test, reset the model and load the latest checkpoint:

# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4037 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


### Save model after training (`SavedModel` format)

In [12]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.5707 - sparse_categorical_accuracy: 0.5137
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4062 - sparse_categorical_accuracy: 0.8892
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.3078 - sparse_categorical_accuracy: 0.9119
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2276 - sparse_categorical_accuracy: 0.9484
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1579 - sparse_categorical_accuracy: 0.9729
INFO:tensorflow:Assets written to: saved_model/my_model/assets


### load model from TF

In [13]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4126 - sparse_categorical_accuracy: 0.8650
Restored model, accuracy: 86.50%
(1000, 10)


## Load model from TFJS

Ref: https://www.tensorflow.org/js/tutorials/conversion/import_saved_model

In [15]:
!pip install tensorflowjs

     |################################| 63 kB 1.6 MB/s eta 0:00:01
     |################################| 103 kB 1.9 MB/s eta 0:00:01


In [18]:
!mkdir -p saved_model/web_model

In [1]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='MobilenetV1/Predictions/Reshape_1' \
    --saved_model_tags=serve \
    saved_model/my_model \
    saved_model/web_model

2021-01-10 16:22:40.302698: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-01-10 16:22:41.598196: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-10 16:22:41.598743: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-10 16:22:41.632631: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-10 16:22:41.632922: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 119.24GiB/s
2021-01-10 16:22:41.632964: I tensorflow/stream_executor/platform/default/ds

In [6]:
# for cat-dog
# from 340MB to 190MB in total
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='cat_dog' \
    --saved_model_tags=serve \
    /learning/ds-sub/mountain-vs-beach-a-z-web/saved_model/cat_dog \
    /learning/ds-sub/mountain-vs-beach-a-z-web/saved_model/cat_dog/tfjs_model

2021-01-10 16:36:32.853707: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-01-10 16:36:34.373950: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-10 16:36:34.374506: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-10 16:36:34.406530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-10 16:36:34.406784: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 119.24GiB/s
2021-01-10 16:36:34.406808: I tensorflow/stream_executor/platform/default/ds